<a href="https://colab.research.google.com/github/andreasslien/machine_learning_group_40/blob/main/Copy_of_Final_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports (General)

In [1]:
# Libraries used throughout notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Can remove random seed here
np.random.seed(42)

In [2]:
# Installing interpret for EBM functionality
!pip install interpret

#Load dataset (General)

In [3]:
# Pandas used to retrieve and represent data
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data",
    header=None)

# Setting names to all columns
df.columns = [
    "ID", "Diagnosis", "Radius_Mean", "Texture_Mean", "Perimeter_Mean", "Area_Mean", "Smoothness_Mean", "Compactness_Mean", "Concavity_Mean", "Concave_points_Mean", "Symmetry_Mean", "Fractal_Dimension_Mean", "Radius_SE", "Texture_SE",
    "Perimeter_SE", "Area_SE", "Smoothness_SE", "Compactness_SE", "Concavity_SE", "Concave_points_SE", "Symmetry_SE", "Fractal_Dimension_SE", "Radius_Worst", "Texture_Worst", "Perimeter_Worst",
    "Area_Worst", "Smoothness_Worst", "Compactness_Worst", "Concavity_Worst", "Concave_points_Worst", "Symmetry_Worst", "Fractal_Dimension_Worst"
]

# Removing columns that have a high correlation
df = df.drop(["Concave_points_Mean","Texture_Worst", "Area_Mean", "Area_Worst", "Perimeter_Mean", "Perimeter_SE", "Radius_Worst"], axis=1)


# Separating attributes from labels
label = df.columns[1]
attributes = df.columns[2:]

y = df[label].apply(lambda x: 0 if x == "B" else 1) #Turning response into 0 and 1 for possible use in EBM
y = y[0:-1]
X = df[attributes]
X = X[0:-1]

In [4]:
print("Attributes:")
print(X)
print("Labels:")
print(y)

Attributes:
     Radius_Mean  Texture_Mean  ...  Symmetry_Worst  Fractal_Dimension_Worst
0          17.99         10.38  ...          0.4601                  0.11890
1          20.57         17.77  ...          0.2750                  0.08902
2          19.69         21.25  ...          0.3613                  0.08758
3          11.42         20.38  ...          0.6638                  0.17300
4          20.29         14.34  ...          0.2364                  0.07678
..           ...           ...  ...             ...                      ...
563        20.92         25.09  ...          0.2929                  0.09873
564        21.56         22.39  ...          0.2060                  0.07115
565        20.13         28.25  ...          0.2572                  0.06637
566        16.60         28.08  ...          0.2218                  0.07820
567        20.60         29.33  ...          0.4087                  0.12400

[568 rows x 23 columns]
Labels:
0      1
1      1
2      1
3   

#Divide into training and test data (General)

In [5]:
# Imported library used
from sklearn.model_selection import train_test_split

np.random.seed(42)

# Splitting into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle = False) # Shuffle is set to false to ensure reproducability


#**EBM**

#Explore the dataset (EBM)

In [6]:
# Imported libraries used
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X, y, name = 'Train Data')
show(hist), # Distribution of data shown

/usr/local/lib/python3.6/dist-packages/interpret/provider/visualize.py:44: UserWarning: Cloud environment detected (['colab', 'ipython']): viz integration is still experimental.
  detected_envs


(None,)

# Make more instances malignant

In [7]:
from interpret.glassbox import ebm as ebm_import
from interpret.glassbox.ebm.ebm import *

class ExplainableBoostingClassifier2(BaseEBM, ClassifierMixin, ExplainerMixin):
    """ Explainable Boosting Classifier. The arguments will change in a future release, watch the changelog. """

    # TODO PK v.3 use underscores here like ClassifierMixin._estimator_type?
    available_explanations = ["global", "local"]
    explainer_type = "model"

    """ Public facing EBM classifier."""

    def __init__(
        self,
        # Explainer
        addition = 0,
        feature_names=None,
        feature_types=None,
        # Preprocessor
        max_bins=255,
        max_interaction_bins=32,
        binning="quantile",
        # Stages
        mains="all",
        interactions=0,
        # Ensemble
        outer_bags=16,
        inner_bags=0,
        # Boosting
        learning_rate=0.01,
        validation_size=0.15,
        early_stopping_rounds=50,
        early_stopping_tolerance=1e-4,
        max_rounds=5000,
        # Trees
        max_leaves=3,
        min_samples_leaf=2,
        # Overall
        n_jobs=-2,
        random_state=42,
    ):
        """ Explainable Boosting Classifier. The arguments will change in a future release, watch the changelog.
        Args:
            feature_names: List of feature names.
            feature_types: List of feature types.
            max_bins: Max number of bins per feature for pre-processing stage.
            max_interaction_bins: Max number of bins per feature for pre-processing stage on interaction terms. Only used if interactions is non-zero.
            binning: Method to bin values for pre-processing. Choose "uniform", "quantile" or "quantile_humanized".
            mains: Features to be trained on in main effects stage. Either "all" or a list of feature indexes.
            interactions: Interactions to be trained on.
                Either a list of lists of feature indices, or an integer for number of automatically detected interactions.
            outer_bags: Number of outer bags.
            inner_bags: Number of inner bags.
            learning_rate: Learning rate for boosting.
            validation_size: Validation set size for boosting.
            early_stopping_rounds: Number of rounds of no improvement to trigger early stopping.
            early_stopping_tolerance: Tolerance that dictates the smallest delta required to be considered an improvement.
            max_rounds: Number of rounds for boosting.
            max_leaves: Maximum leaf nodes used in boosting.
            min_samples_leaf: Minimum number of cases for tree splits used in boosting.
            n_jobs: Number of jobs to run in parallel.
            random_state: Random state.
        """
        super(ExplainableBoostingClassifier2, self).__init__(
            # Explainer
            feature_names=feature_names,
            feature_types=feature_types,
            # Preprocessor
            max_bins=max_bins,
            max_interaction_bins=max_interaction_bins,
            binning=binning,
            # Stages
            mains=mains,
            interactions=interactions,
            # Ensemble
            outer_bags=outer_bags,
            inner_bags=inner_bags,
            # Boosting
            learning_rate=learning_rate,
            validation_size=validation_size,
            early_stopping_rounds=early_stopping_rounds,
            early_stopping_tolerance=early_stopping_tolerance,
            max_rounds=max_rounds,
            # Trees
            max_leaves=max_leaves,
            min_samples_leaf=min_samples_leaf,
            # Overall
            n_jobs=n_jobs,
            random_state=random_state,
            
        )

    # TODO: Throw ValueError like scikit for 1d instead of 2d arrays
    def predict_proba(self, X):
        """ Probability estimates on provided samples.
        Args:
            X: Numpy array for samples.
        Returns:
            Probability estimate of sample for each class.
        """
        check_is_fitted(self, "has_fitted_")
        X_orig, _, _, _ = unify_data(X, None, self.feature_names, self.feature_types)
        X = self.preprocessor_.transform(X_orig)
        X = np.ascontiguousarray(X.T)

        if self.interactions != 0:
            X_pair = self.pair_preprocessor_.transform(X_orig)
            X_pair = np.ascontiguousarray(X_pair.T)
        else:
            X_pair = None

        # TODO PK add a test to see if we handle X.ndim == 1 (or should we throw ValueError)

        prob = EBMUtils.classifier_predict_proba(
            X, X_pair, self.feature_groups_, self.additive_terms_, self.intercept_
        )
        return prob

    def predict(self, X, addition):
        """ Predicts on provided samples.
        Args:
            X: Numpy array for samples.
        Returns:
            Predicted class label per sample.
        """
        check_is_fitted(self, "has_fitted_")
        X_orig, _, _, _ = unify_data(X, None, self.feature_names, self.feature_types)
        X = self.preprocessor_.transform(X_orig)
        X = np.ascontiguousarray(X.T)

        if self.interactions != 0:
            X_pair = self.pair_preprocessor_.transform(X_orig)
            X_pair = np.ascontiguousarray(X_pair.T)
        else:
            X_pair = None

        # TODO PK add a test to see if we handle X.ndim == 1 (or should we throw ValueError)
        return EBMUtils.classifier_predict(
            X,
            X_pair,
            self.feature_groups_,
            self.additive_terms_,
            self.intercept_,
            self.classes_
        )

ebm_import.ebm.ExplainableBoostingClassifier2 = ExplainableBoostingClassifier

In [8]:
from interpret.glassbox import ebm as ebm_import
from interpret.glassbox.ebm.ebm import check_is_fitted
from interpret.utils.all import *
from interpret.glassbox.ebm.utils import *
from sklearn.metrics import confusion_matrix, make_scorer, SCORERS, accuracy_score
from interpret.glassbox import ExplainableBoostingClassifier


def predict_proba2(self, X):
        """ Probability estimates on provided samples.
        Args:
            X: Numpy array for samples.
        Returns:
            Probability estimate of sample for each class.
        """
        check_is_fitted(self, "has_fitted_")
        X_orig, _, _, _ = unify_data(X, None, self.feature_names, self.feature_types)
        X = self.preprocessor_.transform(X_orig)
        X = np.ascontiguousarray(X.T)

        if self.interactions != 0:
            X_pair = self.pair_preprocessor_.transform(X_orig)
            X_pair = np.ascontiguousarray(X_pair.T)
        else:
            X_pair = None

        # TODO PK add a test to see if we handle X.ndim == 1 (or should we throw ValueError)

        prob = EBMUtils.classifier_predict_proba(
            X, X_pair, self.feature_groups_, self.additive_terms_, self.intercept_
        )
        return prob

def predict2(self, X):
        """ Predicts on provided samples.
        Args:
            X: Numpy array for samples.
        Returns:
            Predicted class label per sample.
        """
        check_is_fitted(self, "has_fitted_")
        X_orig, _, _, _ = unify_data(X, None, self.feature_names, self.feature_types)
        X = self.preprocessor_.transform(X_orig)
        X = np.ascontiguousarray(X.T)

        if self.interactions != 0:
            X_pair = self.pair_preprocessor_.transform(X_orig)
            X_pair = np.ascontiguousarray(X_pair.T)
        else:
            X_pair = None

        # TODO PK add a test to see if we handle X.ndim == 1 (or should we throw ValueError)
        return EBMUtils.classifier_predict2(
            X,
            X_pair,
            self.feature_groups_,
            self.additive_terms_,
            self.intercept_,
            self.classes_
        )

def classifier_predict2(X, X_pair, feature_groups, model, intercept, classes):
        log_odds_vector = EBMUtils.decision_function2(
            X, X_pair, feature_groups, model, intercept
        )
        if log_odds_vector.ndim == 1:
            log_odds_vector = np.c_[np.zeros(log_odds_vector.shape), log_odds_vector]
        return classes[np.argmax(log_odds_vector, axis=1)]

def decision_function2(X, X_pair, feature_groups, model, intercept):
        if X.ndim == 1:
            X = X.reshape(X.shape[0], 1)

        # Initialize empty vector for predictions
        if isinstance(intercept, numbers.Number) or len(intercept) == 1:
            score_vector = np.empty(X.shape[1])
        else:
            score_vector = np.empty((X.shape[1], len(intercept)))

        np.copyto(score_vector, intercept)

        # Generate prediction scores
        scores_gen = EBMUtils.scores_by_feature_group(
            X, X_pair, feature_groups, model
        )
        for _, _, scores in scores_gen:
            score_vector += scores

        if not np.all(np.isfinite(score_vector)):  # pragma: no cover
            msg = "Non-finite values present in log odds vector."
            log.error(msg)
            raise Exception(msg)

        counter = 0
        for i in score_vector:
          print('Hello')
          score_vector[counter] = i + 1
          print(1)
          counter += 1
        return score_vector

# ebm_import.ebm.ExplainableBoostingClassifier.predict_proba = predict_proba2
ebm_import.ebm.ExplainableBoostingClassifier.predict2 = predict2
ebm_import.utils.EBMUtils.classifier_predict2 = classifier_predict2
ebm_import.utils.EBMUtils.decision_function2 = decision_function2





In [33]:
def miss_rate(true, predictions):
  confusion_matrix1 = confusion_matrix(true, predictions)
  return 1-confusion_matrix1[1][0]/len(true)

miss_rate_scorer = make_scorer(miss_rate, greater_is_better=True)

In [35]:
from sklearn.model_selection import cross_val_score

                           
np.random.seed(42)

# Create a default valued model model



all_accuracies = []
all_miss_rates = []


print(len(y))

568


In [38]:
# No addition



#base_classifier.fit(X_train, y_train)
#ebm_default_predictions = base_classifier.predict2(X_test, addition = i)


for j in range(7):
  def decision_function2(X, X_pair, feature_groups, model, intercept, addition = j):
        if X.ndim == 1:
            X = X.reshape(X.shape[0], 1)

        # Initialize empty vector for predictions
        if isinstance(intercept, numbers.Number) or len(intercept) == 1:
            score_vector = np.empty(X.shape[1])
        else:
            score_vector = np.empty((X.shape[1], len(intercept)))

        np.copyto(score_vector, intercept)

        # Generate prediction scores
        scores_gen = EBMUtils.scores_by_feature_group(
            X, X_pair, feature_groups, model
        )
        for _, _, scores in scores_gen:
            score_vector += scores

        if not np.all(np.isfinite(score_vector)):  # pragma: no cover
            msg = "Non-finite values present in log odds vector."
            log.error(msg)
            raise Exception(msg)

        counter = 0
        for i in score_vector:
          score_vector[counter] = i + addition
          counter += 1
        return score_vector
  ebm_import.utils.EBMUtils.decision_function = decision_function2

  base_classifier = ExplainableBoostingClassifier(n_jobs = -1, validation_size=0.30, random_state = 42)

  accuracies = cross_val_score(base_classifier, X, y, cv=5).tolist()
  miss_rates = cross_val_score(base_classifier, X, y, cv=5, scoring=miss_rate_scorer).tolist()

  average_accuracy = sum(accuracies)/5
  average_miss_rate = sum(miss_rates)/5

  all_accuracies.append(average_accuracy)
  all_miss_rates.append(average_miss_rate)


print('Accuracy')
print(all_accuracies)
print('Miss rates')
print(all_miss_rates)
  




<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
Accuracy
[0.9597474747474748, 0.9546969696969696, 0.9244444444444444, 0.8992171717171717, 0.8640151515151515, 0.801060606060606, 0.9560006210215806, 0.9595249184909175, 0.9365626455519329, 0.8960409874243129, 0.8536873156342184, 0.7726595249184911, 0.6263778916317342]
Miss rates
[0.9722979797979798, 0.9848989898989899, 0.9899242424242425, 0.992449494949495, 0.9949747474747476, 1.0, 0.9700978108989287, 0.982425089271852, 0.9929669306008384, 0.9929669306008384, 0.9964757025306629, 0.9982456140350877, 1.0]


# Training Randomized Search CV (EBM)

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

np.random.seed(42)

#Max bins
max_bins = [205, 255, 305]
#Learning rate
learning_rate = [0.001, 0.01, 0.1]
#Max rounds
max_rounds = [2500, 5000, 7500]
#Max leaves
max_leaves = [2,3,4]
#Min samples leaf
min_samples_leaf = [2, 3]

# Create the random grid
random_grid = {'max_bins': max_bins,
               'learning_rate': learning_rate,
               'max_rounds': max_rounds,
               'max_leaves': max_leaves,
               'min_samples_leaf': min_samples_leaf,

}
# Create base model for tuning
ebm_randomized_cv = ExplainableBoostingClassifier()

# Random search of parameters, 5-fold CV being used, 
# Searches among 20 iterations
randomized_search_cv = RandomizedSearchCV(estimator = ebm_randomized_cv, param_distributions = random_grid, n_iter = 4, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the model
randomized_search_cv.fit(X_train, y_train)

# Setting ebm_randomized_search_cv_estimator to the estimator that on the highest accuracy on average over the 5 folds
ebm_randomized_search_cv_estimator = randomized_search_cv.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.5min finished


In [13]:
print('Best tried combination of hyperparameters with RSCV:')
print(randomized_search_cv.best_params_)


Best tried combination of hyperparameters with RSCV:
{'min_samples_leaf': 3, 'max_rounds': 2500, 'max_leaves': 2, 'max_bins': 205, 'learning_rate': 0.01}


# Accuracy: default and randomized (EBM)

In [14]:
from sklearn.metrics import accuracy_score

# Using default hyperparameters to predict test set
ebm_default_predictions = ebm_base_search_estimator.predict(X_test)
# Using RSCV hyperparameters to predict test set
ebm_randomized_predictions = ebm_randomized_search_cv_estimator.predict(X_test)


print('Default hyperparameters accuracy:')
print(accuracy_score(y_test, ebm_default_predictions))
print('RSCV hyperparameters accuracy')
print(accuracy_score(y_test, ebm_randomized_predictions))



NameError: ignored

#Confusion matrix (EBM)

In [ ]:
from sklearn.metrics import confusion_matrix

# plot_confusion_matrix used to present confusion matrix with graphics
def plot_confusion_matrix(conf_matrix):
  plt.imshow(conf_matrix,cmap=plt.cm.Blues,interpolation='nearest')
  plt.title('Confusion Matrix without Normalization')
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  plt.grid(b=None)
  tick_marks = np.arange(len(conf_matrix))
  plt.xticks(tick_marks)
  plt.yticks(tick_marks)
  color_threshold = conf_matrix.max() / 2.
  for i in range(len(conf_matrix)):
    for j in range(len(conf_matrix)):
      text_color = 'white' if conf_matrix[i,j] > color_threshold else 'black'
      plt.text(j,i,format(conf_matrix[i,j],'d'), horizontalalignment='center', color=text_color)

# Create the confusion matrix
conf_mat = confusion_matrix(y_test, ebm_default_predictions)
plot_confusion_matrix(conf_mat)

#Global explanations (EBM)

In [ ]:
# By selecting 'Summary' as 'Select Component to Graph' the 15 most important attributes based on mean absolute score of importance are shown
# By selecting an attribute specific information is shown about each attribute

ebm_global = ebm_base_search_estimator.explain_global(name='EBM')
show(ebm_global)

In [ ]:
# visualize present 'Summary' from above

ebm_viz = ebm_global.visualize()
ebm_viz

# Styling of visualize (EBM)

In [ ]:
# Imported libraries used
from interpret.visual import plot
from interpret.visual.plot import plot_horizontal_bar

# The content from plot_horizontal_bar_difference is copied from plot_horizontal_bar at https://github.com/interpretml/interpret/blob/master/python/interpret-core/interpret/visual/plot.py
# A change in the code was necesarry to show a 'Relative importance' as the xtitle

# Copyright (c) 2019 Microsoft Corporation
# Distributed under the MIT software license

COLORS = ["#1f77b4", "#ff7f0e", "#808080"] # Must include different colors

def plot_horizontal_bar_difference(
    data_dict, multiclass=False, title="", xtitle="Relative importance", ytitle="", start_zero=False # Change xtitle from '' to 'Relative importance'
):
    if data_dict.get("scores", None) is None:  # pragma: no cover
        return None
    scores = data_dict["scores"].copy()
    names = data_dict["names"].copy()
    values = data_dict.get("values", None)
    if values is not None:
        values = data_dict["values"].copy()
        names = _names_with_values(names, values)
    if data_dict.get("perf", None) is not None and title == "":
        title_items = []

        predicted = data_dict["perf"]["predicted"]
        actual = data_dict["perf"]["actual"]
        predicted_score = data_dict["perf"]["predicted_score"]
        actual_score = data_dict["perf"]["actual_score"]

        if (
            "meta" in data_dict and "label_names" in data_dict["meta"]
        ):  # Upgraded classification
            label_names = data_dict["meta"]["label_names"]
            predicted = label_names[predicted]
            title_items.append(
                "Predicted ({}): {:.3f}".format(predicted, predicted_score)
            )

            if not np.isnan(actual):
                actual = label_names[actual]
                title_items.append("Actual ({}): {:.3f}".format(actual, actual_score))
        else:  # Regression or old form of classification
            predicted_score = _pretty_number(predicted_score)
            title_items.append("Predicted ({})".format(predicted_score))

            if not np.isnan(actual):
                actual_score = _pretty_number(actual_score)
                title_items.append("Actual ({})".format(actual_score))

        title = " | ".join(title_items)
    if not multiclass:
        # color by positive/negative:
        color = [COLORS[0] if value <= 0 else COLORS[1] for value in scores]
    else:
        color = []
    extra = data_dict.get("extra", None)
    if extra is not None:
        scores.extend(extra["scores"])
        names.extend(extra["names"])
        if values is not None:
            values.extend(extra["values"])
        color.extend([COLORS[2]] * len(extra["scores"]))
    x = scores
    y = names
    traces = []
    if multiclass:
        for index, cls in enumerate(data_dict["meta"]["label_names"]):
            trace_scores = [x[index] for x in data_dict["scores"]] + [
                data_dict["extra"]["scores"][0][index]
            ]
            trace_names = data_dict["names"] + [data_dict["extra"]["names"]]
            traces.append(
                go.Bar(y=trace_names, x=trace_scores, orientation="h", name=cls)
            )
    else:
        traces.append(go.Bar(x=x, y=y, orientation="h", marker=dict(color=color)))

    if start_zero:
        x_range = [0, np.max(x)]
    else:
        max_abs_x = np.max(np.abs(x))
        if multiclass:
            max_abs_x = np.sum(np.array(x), axis=1)
        x_range = [-max_abs_x, max_abs_x]
    layout = dict(
        title=title,
        yaxis=dict(automargin=True, title=ytitle),
        xaxis=dict(range=x_range, title=xtitle),
    )
    if multiclass:
        layout["barmode"] = "relative"
    figure = go.Figure(data=traces, layout=layout)
    return figure

plot.plot_horizontal_bar_difference = plot_horizontal_bar_difference

# Top ranked attributes sorted by relative importance (EBM)

In [ ]:
# Imported libraries used
from interpret.glassbox import ebm as ebm_import
import plotly.graph_objs as go

# The content from visualize_all_items is copied from visualize at https://github.com/interpretml/interpret/blob/master/python/interpret-core/interpret/glassbox/ebm/ebm.py
# A change in the code was necesarry to show a complete ranking
def visualize_all_items(self, key=None):
        """ Provides interactive visualizations.
        Args:
            key: Either a scalar or list
                that indexes the internal object for sub-plotting.
                If an overall visualization is requested, pass None.
        Returns:
            A Plotly figure.
        """
        """
        from ...visual.plot import (
            plot_continuous_bar,
            plot_horizontal_bar,
            sort_take,
            is_multiclass_global_data_dict,
        )
        """
        
        data_dict = self.data(key)
        if data_dict is None:
            return None

        # Overall graph
        if self.explanation_type == "global" and key is None:
            data_dict = plot.sort_take(
                data_dict, sort_fn=lambda x: -abs(x), top_n=23, reverse_results=True # Change top_n from 15 to 23, to show a complete ranking
            )
            total = 0
            for i in data_dict['scores']:
              total += i
            
            data_dict['scores'] = [x / total for x in data_dict['scores']]
            figure = plot_horizontal_bar_difference(
                data_dict,
                title="",
                start_zero=True,
            )

            return figure

        # Continuous feature graph
        if (
            self.explanation_type == "global"
            and self.feature_types[key] == "continuous"
        ):
            title = self.feature_names[key]
            if is_multiclass_global_data_dict(data_dict):
                figure = plot_continuous_bar(
                    data_dict, multiclass=True, show_error=False, title=title
                )
            else:
                figure = plot_continuous_bar(data_dict, title=title)

            return figure

        return super().visualize(key)

# Adding the new function to allow further use of visualize_all_items on an EBM global model
ebm_import.ebm.EBMExplanation.visualize_all_items = visualize_all_items

ebm_global = ebm_base_search_estimator.explain_global(name='EBM')
ebm_global.visualize_all_items()


#Local explanations (EBM)

In [ ]:
# By selecting a row in the scroll down bar it is possible to see how different attributes affected individual classifications

ebm_local = ebm_base_search_estimator.explain_local(X_test, y_test, name='EBM')
show(ebm_local)
